In [1]:
import torch
import torch.onnx
import torchvision
from torchvision import datasets, models, transforms
from torch.autograd import Variable
import util

In [2]:
from session import *
from LR_Schedule.cos_anneal import CosAnneal
from LR_Schedule.lr_find import lr_find
from callbacks import *
from validation import *
import Datasets.ImageData as ImageData
from Transforms.ImageTransforms import *

In [3]:
import onnx
import onnx_caffe2.backend

ModuleNotFoundError: No module named 'caffe2'

In [ ]:
num_classes = 8

In [ ]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Sequential(
    nn.Dropout(p=.5),
    nn.Linear(num_ftrs, num_classes),
    nn.Sigmoid()
)

In [ ]:
criterion = nn.BCELoss()
optim_fn = optim.Adam
sess = Session(model_ft, criterion, optim_fn, 1e-2)

In [ ]:
sess.load('TrainLayer1')

In [ ]:
torch.save(sess.model, "./torch-model")

In [ ]:
imsize = 224
dummy_input = Variable(util.to_gpu(torch.randn(1, 3, 224, 224)))

In [ ]:
input_names = ["image"]
output_names = ["classification"]

In [ ]:
with EvalModel(sess.model):
    torch_out = torch.onnx._export(sess.model,             # model being run
                               dummy_input,                       # model input (or a tuple for multiple inputs)
                               "roaddamage.onnx", # where to save the model (can be a file or file-like object)
                               export_params=True)      # store the trained parameter weights inside the model file


In [ ]:
# Load the ONNX ModelProto object. model is a standard Python protobuf object
model = onnx.load("roaddamage.onnx")

# prepare the caffe2 backend for executing the model this converts the ONNX model into a
# Caffe2 NetDef that can execute it. Other ONNX backends, like one for CNTK will be
# availiable soon.
prepared_backend = onnx_caffe2.backend.prepare(model)

# run the model in Caffe2

# Construct a map from input names to Tensor data.
# The graph of the model itself contains inputs for all weight parameters, after the input image.
# Since the weights are already embedded, we just need to pass the input image.
# Set the first input.
W = {model.graph.input[0].name: dummpy_input.data.numpy()}

# Run the Caffe2 net:
c2_out = prepared_backend.run(W)[0]

# Verify the numerical correctness upto 3 decimal places
np.testing.assert_almost_equal(torch_out.data.cpu().numpy(), c2_out, decimal=3)

print("Exported model has been executed on Caffe2 backend, and the result looks good!")
